In [30]:
import pandas as pd

df = pd.read_csv("policy_zscores_labeled.csv")

# Show column names clearly
for col in df.columns:
    print(f"'{col}'")

'Date'
'Federal Budget Deficit (% GDP)'
'Fed Balance Sheet MoM %'
'Fed Funds Rate (%)'
'Core CPI (MoM %)'
'Real Fed Funds Rate (%)'
'Fed Funds Rate (%).1'
'Federal Budget Deficit (% GDP)_Label'
'Fed Balance Sheet MoM %_Label'
'Fed Funds Rate (%)_Label'
'Core CPI (MoM %)_Label'
'Real Fed Funds Rate (%)_Label'
'Fed Funds Rate (%).1_Label'


In [32]:
df.columns = df.columns.str.strip()

df.rename(columns={
    'Fed Funds Rate (%)': 'FedFunds_Z',
    'Real Fed Funds Rate (%)': 'RealFedFunds_Z',
    'Fed Balance Sheet MoM %': 'BalanceSheet_Z',
    'Federal Budget Deficit (% GDP)': 'FiscalDeficit_Z'
}, inplace=True)


In [34]:
def normalized_contribution(z, weight, cap=2.0):
    z = max(min(z, cap), -cap)
    scaled = (z + cap) / (2 * cap)
    return scaled * weight


In [36]:
df['PolicyRate_SubTheme_Score'] = df['FedFunds_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100
df['RealRate_SubTheme_Score'] = df['RealFedFunds_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100
df['BalanceSheet_SubTheme_Score'] = df['BalanceSheet_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100
df['Fiscal_SubTheme_Score'] = df['FiscalDeficit_Z'].apply(lambda z: normalized_contribution(z, 1.0)) * 100


In [42]:
weights_policy = {
    'PolicyRate_SubTheme_Score': 0.3,
    'RealRate_SubTheme_Score': 0.3,
    'BalanceSheet_SubTheme_Score': 0.2,
    'Fiscal_SubTheme_Score': 0.2
}

df['Policy_Theme_Score'] = sum(
    df[col] * weight for col, weight in weights_policy.items()
).round(2)


In [44]:
print(df[['Date', 'Policy_Theme_Score']].head())


         Date  Policy_Theme_Score
0  2004-01-01               46.47
1  2004-02-01               46.01
2  2004-03-01               46.28
3  2004-04-01               47.29
4  2004-05-01               46.78


In [46]:
df[['Date',
   'PolicyRate_SubTheme_Score',
    'RealRate_SubTheme_Score',
    'BalanceSheet_SubTheme_Score',
    'Fiscal_SubTheme_Score',
    'Policy_Theme_Score',
]].to_csv("policy_theme_output.csv", index=False)